In [1]:
# esta celda controla el estilo del cuaderno
from IPython.core.display import HTML
def css_styling():
    styles = open("custom.css", "r").read()
    return HTML(styles)
css_styling()

In [ ]:
%%capture

!pip install docarray
!pip install sentence_transformers
!pip install seaborn

# Modelos de Lenguaje y Similitud Semántica

En esta presentación vamos a continuar trabajando con modelos de lenguaje ya pre-entrenados, intentando entender cómo funcionan y viendo ejemplos concretos de uso.


## HuggingFace

[HuggingFace](https://huggingface.co) se ha convertido en todo un reference en el mundo del PLN. 

- fueron los primeros en distribuir una versión de BERT compatible con PyTorch.
- crearon un interfaz interoperable entre TensorFlow y PyTorch.
- su librería `transformers` permite acceder a decenas de modelos con arquitecturas diferentes con una interfaz común.
- su librería `datasets` permite acceder a decenas de datasets e interaccionar con nuestros propios datos de manera sencilla 
- han creado un repositorio de modelos de lenguaje

Vamos a recorrer uno de sus principales recursos: la [central de modelos](https://huggingface.co/models), un repositorio público de modelos de lenguaje ya preentrenados.

DistilGPT-2 es una versión de tamaño reducido del célebre modelo [GPT-2](https://openai.com/blog/better-language-models/), desarrollado por OpenAI en 2019.

In [ ]:
import numpy as np 
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
gpt2_tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
gpt2_model = AutoModelForCausalLM.from_pretrained("distilgpt2")

gpt2_model.to(device)
gpt2_model.eval()

In [ ]:
text = """The US president announced earlier today"""
 
input_ids = torch.tensor(gpt2_tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
 
outputs = gpt2_model.generate(
    input_ids.to(device), 
    max_length=50,
    do_sample=True,
    top_k=20,
    temperature=0.7
    )
 
print(gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True))
# outputs.shape,outputs[0].shape

[GPT-3](https://openai.com/blog/gpt-3-apps/) es la última versión de este modelo de lenguaje masivo desarrollado por OpenAI, publicada en 2021. [No está disponible en abierto](https://gpt3-openai.com/), pero un grupo de investigadores llamado [Eleuther AI](https://www.eleuther.ai/) ha entrenado una versión alternativa por su cuenta.

In [ ]:
gpt3_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
gpt3_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

gpt3_model.to(device)
gpt3_model.eval()

In [ ]:
text = """The US president announced earlier today"""
 
input_ids = torch.tensor(gpt3_tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
 
outputs = gpt3_model.generate(
    input_ids.to(device), 
    max_length=50,
    do_sample=True,
    top_k=20,
    temperature=0.7
    )
 
print(gpt3_tokenizer.decode(outputs[0], skip_special_tokens=True))
# outputs.shape,outputs[0].shape

## Modelos de similitud semántica

El model hub de HF dispone de una buena variedad de [modelos de lenguaje ajustados para tareas de similitud semántica](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads).

In [ ]:
from sentence_transformers import SentenceTransformer

ss_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
embedding = ss_model.encode("I love tacos and micheladas")

print(embedding.shape)
print(embedding[:20])

In [ ]:
embedding = ss_model.encode(["I don't like it", "I like it"])
print(embedding.shape)

In [ ]:
messages = [
    # Smartphones
    "I like my phone",
    "My phone is not good.",
    "Your cellphone looks great.",

    # Weather
    "Will it snow tomorrow?",
    "Recently a lot of hurricanes have hit the US",
    "Global warming is real",

    # Food and health
    "An apple a day, keeps the doctors away",
    "Eating strawberries is healthy",
    "Is paleo better than keto?",

    # Asking about age
    "How old are you?",
    "what is your age?",
    "Am I younger than you?",
]

embeddings_en = ss_model.encode(messages)

Una vez codificados nuestros textos de ejemplo como vectores, vamos a crear una base de datos de embeddings de manera que podamos lanzar búsquedas basadas en similitud semántica. Para ello, vamos a aprovechar la librería [DocArray](https://docarray.jina.ai/) de [Jina.ai](https://jina.ai), otra empresa de la que merece estar al tanto.

In [ ]:
from docarray import DocumentArray, Document

da = DocumentArray([Document(text=m, embedding=embeddings_en[i]) for i, m in enumerate(messages)])

In [ ]:
def similarity_search(
    q: str, 
    model: SentenceTransformer = ss_model, 
    da: DocumentArray = da,
    n: int = 3
) -> None:
    query = (
        Document(text=q, embedding=model.encode(q))
        .match(da, limit=n, exclude_self=True, metric="cosine")
        )

    return [(m.text, m.scores["cosine"].value) for m in query.matches]


In [ ]:
similarity_search(q="My handheld device has two screens")

In [ ]:
similarity_search(q="Mostly dry. Freeze-thaw conditions (max 50°F on Mon afternoon, min 32°F on Tue night).")

In [ ]:
similarity_search(q="I'm 40 years old")

In [ ]:
import seaborn as sns

def plot_similarity(labels, features):
    """Dibuja un mapa de calor con las distancias de similitud"""
    corr = np.inner(features, features)
    sns.set(font_scale=1.2)
    g = sns.heatmap(
        corr,
        xticklabels=labels,
        yticklabels=labels,
        vmin=0,
        vmax=1,
        cmap="YlOrRd")
    g.set_xticklabels(labels, rotation=90)
    g.set_title("Similitud Semántica")

In [ ]:
plot_similarity(messages, embeddings_en)

In [ ]:
ss_model_es = SentenceTransformer("hiiamsid/sentence_similarity_spanish_es")

In [ ]:
mensajes = [
    "Me gusta mi teléfono.",
    "Mi móvil no es muy bueno.",
    "Tu teléfono parece caro.",

    "¿Va a nevar mañana?",
    "Ha habido muchos huracanes en EEUU este año",
    "El calentamiento global es real",

    "El que come una manzana aleja al médico de su cama",
    "Comer fruta y verdura es muy saludable",
    "¿Es mejor hacer dieta paleo o keto?",

    "¿Cuántos años tienes?",
    "¿Cuál es tu edad?",
    "¿Soy más joven que usted o más viejo?",
]

embeddings_es = ss_model_es.encode(mensajes)
da_es = DocumentArray([Document(text=m, embedding=embeddings_es[i]) for i, m in enumerate(mensajes)])

In [ ]:
print(embeddings_es.shape)

In [ ]:
similarity_search(q="Samsung, Xiaomi y Apple se reparten el mercado", model=ss_model_es, da=da_es)

In [ ]:
similarity_search(q="No soy un niñato, tengo más de 40 palos", model=ss_model_es, da=da_es)

In [ ]:
similarity_search(q="Podría comer gazpacho y tortilla de patatas todos los días", model=ss_model_es, da=da_es)

In [ ]:
plot_similarity(mensajes, embeddings_es)

## Universal Sentence Encoder

El ecosistema TensorFlow tiene su propio repositorio público de modelos: [TFHub](https://tfhub.dev). Hay varios modelos, pero probablemente uno de los más interesantes es [Universal Sentence Encoder (USE)](https://tfhub.dev/google/universal-sentence-encoder/4).

Al contrario que los anteriores, estos modelos de lenguaje no están basado en la arquitectura de Transformer, sino que utiliza otro tipo de red neuronal llamado *Deep Averaging Network* (DAN).

In [ ]:
%%capture

!pip install tensorflow tensorflow_hub tensorflow_text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [ ]:
USE_URL = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_model = hub.load(USE_URL)

In [ ]:
e = use_model(["hello"])
e[0].shape

In [ ]:
use_embeddings = use_model(messages)
print(use_embeddings.shape)

use_da = DocumentArray([Document(text=m, embedding=use_embeddings[i]) for i, m in enumerate(messages)])

In [ ]:
def use_similarity_search(
    q: str, 
    model, 
    da,
    n: int = 3
) -> None:
    query = (
        Document(text=q, embedding=model([q])[0])
        .match(da, limit=n, exclude_self=True, metric="cosine")
        )

    return [(m.text, m.scores["cosine"].value) for m in query.matches]

In [ ]:
use_similarity_search(q="My handheld device has two screens", model=use_model, da=use_da)

In [ ]:
use_similarity_search(q="Mostly dry. Freeze-thaw conditions (max 50°F on Mon afternoon, min 32°F on Tue night).", model=use_model, da=use_da)

In [ ]:
use_similarity_search(q="I'm 40 years old", model=use_model, da=use_da)

In [ ]:
plot_similarity(messages, use_embeddings)

¿Qué tal funciona USE en español? No hay un modelo monolingüe específico, pero sí [existe una variante multilingüe que se ha entrenado en varios idiomas](https://tfhub.dev/google/universal-sentence-encoder-multilingual/3). Probemos uno de ellos.

In [ ]:
MULTI_USE_URL = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
multi_use_model = hub.load(MULTI_USE_URL)

In [ ]:
use_embeddings_es = multi_use_model(mensajes)
print(use_embeddings_es.shape)

use_da_es = DocumentArray([Document(text=m, embedding=use_embeddings_es[i]) for i, m in enumerate(mensajes)])

In [ ]:
use_similarity_search(q="Samsung, Xiaomi y Apple se reparten el mercado", model=multi_use_model, da=use_da_es)

In [ ]:
use_similarity_search(q="No soy un niñato, tengo más de 40 palos", model=multi_use_model, da=use_da_es)

In [ ]:
use_similarity_search(q="Podría comer gazpacho y tortilla de patatas todos los días", model=multi_use_model, da=use_da_es)

In [ ]:
plot_similarity(mensajes, use_embeddings_es)